# Two-Class Support Vector Machine exercise

Complete and hand in this completed worksheet with your assignment submission.

In this exercise you will:
    
- Understand the logic of the following code to use SVM for image classification.
- Implement a linear SVM by calling functions from scikit-learn module.
- Tune parameters of SVM. Analyze the tuned results.

In [ ]:
!wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz -O cifar-10-python.tar.gz
!tar -xzvf cifar-10-python.tar.gz
!rm cifar-10-python.tar.gz

### Prepare two-class data
Prepare CIFAR-10 images. Note that we only use two classes (cat, dog) here for image classification.

In [ ]:
# define functions for loading CIFAR-10.

from six.moves import cPickle as pickle
import numpy as np
import os
import platform

def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
        return X, Y

def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [ ]:
# Load the raw CIFAR-10 data.
cifar10_dir = './cifar-10-batches-py'

# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
# Extract the two class images. Separate the data into training and validation set.

train_smaple_num = 100
test_smaple_num = 100

# label of 'cat' and 'dog'
selected_label = [3, 5]

# Re-split the 'cat' and 'dog' samples
cat_mask_train = y_train == selected_label[0]
dog_mask_train = y_train == selected_label[1]

X2_train_p1 = X_train[cat_mask_train][:train_smaple_num]
y2_train_p1 = np.zeros(X2_train_p1.shape[0], dtype=np.int64)
X2_train_p2 = X_train[dog_mask_train][:train_smaple_num]
y2_train_p2 = np.ones(X2_train_p2.shape[0], dtype=np.int64)
X2_train = np.concatenate([X2_train_p1, X2_train_p2], axis=0)
y2_train = np.concatenate([y2_train_p1, y2_train_p2], axis=0)

cat_mask_test = y_test == selected_label[0]
dog_mask_test = y_test == selected_label[1]

X2_test_p1 = X_test[cat_mask_test][:test_smaple_num]
y2_test_p1 = np.zeros(X2_test_p1.shape[0], dtype=np.int64)
X2_test_p2 = X_test[dog_mask_test][:test_smaple_num]
y2_test_p2 = np.ones(X2_test_p2.shape[0], dtype=np.int64)
X2_test = np.concatenate([X2_test_p1, X2_test_p2], axis=0)
y2_test = np.concatenate([y2_test_p1, y2_test_p2], axis=0)

# Double check the size of the training and test data.
print('Training data shape: ', X2_train.shape)
print('Training labels shape: ', y2_train.shape)
print('Test data shape: ', X2_test.shape)
print('Test labels shape: ', y2_test.shape)

### Extract feature vectors
We utilized a pretrained CNN to extract the features vectors. Please understand the logic of following code.

In [ ]:
# Extract the feature vectors of images.
# We utilized a pretrained ResNet18 network to extract the layer4 features as the sample features.

import torch
import torch.nn as nn
from torchvision import models

class ResBase18(nn.Module):
    def __init__(self):
        super(ResBase18, self).__init__()
        # model_resnet18 = models.resnet18(pretrained=True)
        model_resnet18 = models.resnet18(pretrained=True)
        self.conv1 = model_resnet18.conv1
        self.bn1 = model_resnet18.bn1
        self.relu = model_resnet18.relu
        self.maxpool = model_resnet18.maxpool
        self.layer1 = model_resnet18.layer1
        self.layer2 = model_resnet18.layer2
        self.layer3 = model_resnet18.layer3
        self.layer4 = model_resnet18.layer4
        self.avgpool = model_resnet18.avgpool

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x

# define a instance of network, turn to evaluation mode.
model = ResBase18()
model.eval()

# extract features of training and test samples.
with torch.no_grad():
    X2_train_feat = model(torch.FloatTensor(X2_train).permute(0, 3, 1, 2))
    X2_test_feat = model(torch.FloatTensor(X2_test).permute(0, 3, 1, 2))
X2_train_feat = X2_train_feat.numpy()
X2_test_feat = X2_test_feat.numpy()

# Check the dimension of extracted features
print('Training feature shape: ', X2_train_feat.shape)
print('Testing feature shape: ', X2_test_feat.shape)

### Implement a Linear Support Vector Machine(SVM)
Check the documentation at https://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm to see how to use these functions to train and test a Support Vector Machine(SVM).  
**TODO**: 1) implement a linear SVM; 2) make predictions with your SVM.

In [ ]:
from sklearn import svm

########################################################################################################
# Step 1 TODO:                                                                                         #
# Training a linear SVM using training set.                                                            #
# You need to write code that trains a SVM. Please refer to the document of sklearn.svm.               #
########################################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Create a svm instance
svc =

# Train the svm instance svc with the training set samples 'X2_train_feat' and labels 'y2_train' as input


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


########################################################################################################
# Step 2 TODO:                                                                                         #
# Using the trained SVM to predict the classification results of validation set.                       #
# You need to write code that test the SVM you implemented above.                                      #
# Again you may refer to the document to see if any functions are already defined for prediction.      #
########################################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Test our trained svm instance svc with the test set samples 'X2_test_feat'
y2_pred =

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Print the predicted outputs and accuracies.
# We use precision, recall and f-measure to measure the accuracy of image classification.
# To understand the meaning of these metrics, you can refer to
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html.
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y2_test,y2_pred))

#### Tune paramaters and analyze the results
**TODO**: Document the parameters you have tried such as the kernel function or the penalty dunction, and write down your analysis.

In [ ]:

########################################################################################################
# Step 3 TODO:                                                                                         #
# You may tune parameters like kernels and penality functions to improve your testing results.         #
# Then, you need to write down what parameters you have tried and the analysis of results.             #
# Note that you can write your analysis in the next block or in the writing assignment.                #
########################################################################################################


Analysis:
